In [52]:
import os
import sys
module_path = os.path.abspath(os.path.join('../../..'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [53]:
print(module_path)

/Users/revilooliver/Documents/quantum_computing/research/pauli_sandwitching/cut4mitigation


In [93]:
# author: Ji Liu email: ji.liu@anl.gov

import itertools, numpy
import circuit_cutter
import mlrecon_methods as ml
import numpy as np
import mthree

import qiskit
import qiskit.providers.aer.noise as noise
from qiskit import *
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, Aer, execute, transpile

from qiskit.transpiler import PassManager

from qiskit.converters import circuit_to_dag
from qiskit.visualization import dag_drawer, plot_histogram
from qiskit.compiler import assemble

from qiskit.tools.monitor import job_monitor, backend_monitor, backend_overview

from qiskit.providers.aer.noise import NoiseModel

from utils.utils import filter_results, dict_to_list, H_distance, total_counts, norm_dict
from vqe_utils import read_from_file, MeasureCircuit, find_commute_groups, evaluation
from mlrecon_methods import run_circuits, collect_fragment_circuits, organize_tomography_data_from_list

In [55]:
qiskit.__qiskit_version__

{'qiskit-terra': '0.22.2', 'qiskit-aer': '0.11.1', 'qiskit-ignis': '0.7.1', 'qiskit-ibmq-provider': '0.19.2', 'qiskit': '0.39.2', 'qiskit-nature': '0.3.0', 'qiskit-finance': None, 'qiskit-optimization': None, 'qiskit-machine-learning': None}

In [56]:
numpy.set_printoptions(linewidth = 200)

qubits = 5

simulation_backend = "qasm_simulator"

seed = 0
print_circuits = True

In [57]:
IBMQ.load_account()
provider = IBMQ.get_provider(hub='ibm-q-ornl', group='ornl', project='chm185')
provider.backends()

ibmqfactory.load_account:WARNING:2022-11-18 14:31:45,908: Credentials are already in use. The existing account in the session will be replaced.


[<IBMQSimulator('ibmq_qasm_simulator') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_montreal') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_kolkata') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_mumbai') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_lima') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_belem') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_quito') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQBackend('ibmq_guadalupe') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_statevector') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>,
 <IBMQSimulator('simulator_mps') from IBMQ(hub='ibm-q-ornl', grou

In [58]:
device = provider.get_backend('ibmq_toronto')
device

<IBMQBackend('ibmq_toronto') from IBMQ(hub='ibm-q-ornl', group='ornl', project='chm185')>

In [59]:
device.configuration().to_dict()['max_shots']

32000

In [60]:
from qiskit import IBMQ, Aer
from qiskit.providers.aer.noise import NoiseModel
noise_model = NoiseModel.from_backend(device, gate_error = False, thermal_relaxation= False)
print(noise_model)

NoiseModel:
  Basis gates: ['cx', 'id', 'reset', 'rz', 'sx', 'x']
  Instructions with noise: ['measure']
  Qubits with noise: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26]
  Specific qubit errors: [('measure', (0,)), ('measure', (1,)), ('measure', (2,)), ('measure', (3,)), ('measure', (4,)), ('measure', (5,)), ('measure', (6,)), ('measure', (7,)), ('measure', (8,)), ('measure', (9,)), ('measure', (10,)), ('measure', (11,)), ('measure', (12,)), ('measure', (13,)), ('measure', (14,)), ('measure', (15,)), ('measure', (16,)), ('measure', (17,)), ('measure', (18,)), ('measure', (19,)), ('measure', (20,)), ('measure', (21,)), ('measure', (22,)), ('measure', (23,)), ('measure', (24,)), ('measure', (25,)), ('measure', (26,))]


In [61]:
noisy_simulator = provider.get_backend('ibmq_qasm_simulator')

In [62]:
front_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_front.qasm")

In [63]:
mid_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_mid.qasm")

In [64]:
end_layer = QuantumCircuit.from_qasm_file("qasm/vqe_Lih_end.qasm")

In [65]:
orign_circuit = front_layer + mid_layer + end_layer

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69609/743480212.py:1: DeprecationWarning: The QuantumCircuit.__add__() method is being deprecated.Use the compose() method which is more flexible w.r.t circuit register compatibility.
  orign_circuit = front_layer + mid_layer + end_layer


In [66]:
orign_circuit.draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├──────────────────────────────────────────────■────────»
     └───────────────┘                                                       »
q_4: ────────────────────────────────────────────────────────────────────────»
                                                                             »
«                      
«q_0: ─────────────────
«                      
«q_1: ─────────────────
«     ┌───────────────┐
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └──────────────┘
«q_4: ─────────────────
«

In [67]:
#checking circuit

In [68]:
def apply_checking_circuit(qc, ctrl_bits, ancilla_bits, side = None):
    if len(ctrl_bits) != len(ancilla_bits):
        print("Size mismatch")
        return None
    if side == 'front':
        for i in ancilla_bits:
            qc.h(i)
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
    elif side == 'end':
        for j,k in zip(ctrl_bits, ancilla_bits):
            qc.cz(j, k)
        for i in ancilla_bits:
            qc.h(i)
    else:
        print("Side undefined")

In [69]:
pauli_check_circs = []
for i in range(0, 4):
    circuit = front_layer.copy()
    apply_checking_circuit(circuit, [i], [4], side = 'front')
    circuit += mid_layer
    apply_checking_circuit(circuit, [i], [4], side = 'end')
    circuit += end_layer
    pauli_check_circs.append(circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69609/3323928608.py:5: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += mid_layer
/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69609/3323928608.py:7: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  circuit += end_layer


In [70]:
pauli_check_circs[-1].draw()

┌───┐      ┌───────────────┐   ┌────────────────┐                 »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├─────────────────»
           ├───┤      ├───────────────┤ │ └────────────────┘┌───────────────┐»
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────┤ Ry(-0.015751) ├»
     ┌─────┴───┴─────┐└───────────────┘           │         └───────────────┘»
q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────────────■────────»
     ├───────────────┤                                              │        »
q_3: ┤ Ry(-0.019851) ├────────■─────────────────────────────────────■────────»
     └─────┬───┬─────┘        │                                              »
q_4: ──────┤ H ├──────────────■──────────────────────────────────────────────»
           └───┘                                                             »
«                                      
«q_0: ─────────────────────────────────
«                                      
«q_1: ─────────────────────────────────
«     ┌───────────────┐                
«q_2: ┤ Ry(0.0036205) ├────────────────
«     └───────────────┘┌──────────────┐
«q_3: ────────■────────┤ Ry(0.019873) ├
«             │        └────┬───┬─────┘
«q_4: ────────■─────────────┤ H ├──────
«                           └───┘

In [71]:
pauli_list = read_from_file('LiH_hamiltonian.txt')
pauli_commute = find_commute_groups(pauli_list)

In [72]:
#generate individual measurment circuits that changes the basis
meas_qcs = []
for i in range(0, len(pauli_commute)):
    temp_qc = MeasureCircuit(pauli_commute[i], num_qubits = 4, num_qargs = 5)
    meas_qcs.append(temp_qc)
    print(temp_qc)

YYYY
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZZ
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
                     
q_3: ────────────────
                     
q_4: ────────────────
                     
ZZYY
                     
q_0: ────────────────
                     
q_1: ────────────────
     ┌──────────────┐
q_2: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_3: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_4: ────────────────
                     
YYZX
     ┌──────────────┐
q_0: ┤ U(π/2,0,π/2) ├
     ├──────────────┤
q_1: ┤ U(π/2,0,π/2) ├
     └──────────────┘
q_2: ────────────────
      ┌────────────┐ 
q_3: ─┤ U(π/2,0,π) ├─
      └────────────┘ 
q_4: ────────────────
                     
ZXYY
       

In [73]:
pauli_check_circs[1].draw()

┌───┐      ┌───────────────┐      ┌────────────────┐      »
q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├────■─┤ Ry(-0.0036834) ├──────»
           ├───┤      ├───────────────┤    │ └────────────────┘      »
q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■──■─────────■─────────────■─»
     ┌─────┴───┴─────┐└───────────────┘ │            │             │ »
q_2: ┤ Ry(0.0038243) ├──────────────────┼────────────■──────────■──┼─»
     ├───────────────┤                  │                       │  │ »
q_3: ┤ Ry(-0.019851) ├──────────────────┼───────────────────────■──┼─»
     └─────┬───┬─────┘                  │                          │ »
q_4: ──────┤ H ├────────────────────────■──────────────────────────■─»
           └───┘                                                     »
«                      
«q_0: ─────────────────
«     ┌───────────────┐
«q_1: ┤ Ry(-0.015751) ├
«     ├───────────────┤
«q_2: ┤ Ry(0.0036205) ├
«     └┬──────────────┤
«q_3: ─┤ Ry(0.019873) ├
«      └────┬───┬─────┘
«q_4: ──────┤ H ├──────
«           └───┘

In [74]:
circuit_list = []
for check_circ in pauli_check_circs:
    for meas in meas_qcs:
        temp_circuit = check_circ.copy()
        temp_circuit += meas
        temp_circuit.measure_all()
        circuit_list.append(temp_circuit)

/var/folders/s4/3cvgkkn17hg628pzrzzjnhww0000gn/T/ipykernel_69609/1111001334.py:5: DeprecationWarning: The QuantumCircuit.__iadd__() method is being deprecated. Use the compose() (potentially with the inplace=True argument) and tensor() methods which are more flexible w.r.t circuit register compatibility.
  temp_circuit += meas


In [75]:
len(circuit_list)

100

In [76]:
circuit_list[2].draw()

┌───┐      ┌───────────────┐            ┌────────────────┐»
   q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■──■─────■─┤ Ry(-0.0036834) ├»
              ├───┤      ├───────────────┤ │  │     │ ├───────────────┬┘»
   q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─┼──■──■──┼─┤ Ry(-0.015751) ├─»
        ┌─────┴───┴─────┐└───────────────┘ │     │  │ └───────────────┘ »
   q_2: ┤ Ry(0.0038243) ├──────────────────┼─────■──┼─────────■─────────»
        ├───────────────┤                  │        │         │         »
   q_3: ┤ Ry(-0.019851) ├──────────────────┼────────┼─────────■─────────»
        └─────┬───┬─────┘                  │        │       ┌───┐       »
   q_4: ──────┤ H ├────────────────────────■────────■───────┤ H ├───────»
              └───┘                                         └───┘       »
meas_0: ════════════════════════════════════════════════════════════════»
                                                                        »
meas_1: ════════════════════════════════════════════════════════════════»
                                                                        »
meas_2: ════════════════════════════════════════════════════════════════»
                                                                        »
meas_3: ════════════════════════════════════════════════════════════════»
                                                                        »
meas_4: ════════════════════════════════════════════════════════════════»
                                                                        »
«                                          ░ ┌─┐            
«   q_0: ──────────────────────────────────░─┤M├────────────
«                                          ░ └╥┘┌─┐         
«   q_1: ──────────────────────────────────░──╫─┤M├─────────
«        ┌───────────────┐┌──────────────┐ ░  ║ └╥┘┌─┐      
«   q_2: ┤ Ry(0.0036205) ├┤ U(π/2,0,π/2) ├─░──╫──╫─┤M├──────
«        └┬──────────────┤├──────────────┤ ░  ║  ║ └╥┘┌─┐   
«   q_3: ─┤ Ry(0.019873) ├┤ U(π/2,0,π/2) ├─░──╫──╫──╫─┤M├───
«         └──────────────┘└──────────────┘ ░  ║  ║  ║ └╥┘┌─┐
«   q_4: ──────────────────────────────────░──╫──╫──╫──╫─┤M├
«                                          ░  ║  ║  ║  ║ └╥┘
«meas_0: ═════════════════════════════════════╩══╬══╬══╬══╬═
«                                                ║  ║  ║  ║ 
«meas_1: ════════════════════════════════════════╩══╬══╬══╬═
«                                                   ║  ║  ║ 
«meas_2: ═══════════════════════════════════════════╩══╬══╬═
«                                                      ║  ║ 
«meas_3: ══════════════════════════════════════════════╩══╬═
«                                                         ║ 
«meas_4: ═════════════════════════════════════════════════╩═
«

In [77]:
circuit_list[-2].draw()

┌───┐      ┌───────────────┐   ┌────────────────┐»
   q_0: ──────┤ X ├──────┤ Ry(-0.003534) ├─■─┤ Ry(-0.0036834) ├»
              ├───┤      ├───────────────┤ │ └────────────────┘»
   q_1: ──────┤ X ├──────┤ Ry(-0.015869) ├─■─────────■─────────»
        ┌─────┴───┴─────┐└───────────────┘           │         »
   q_2: ┤ Ry(0.0038243) ├────────────────────────────■─────────»
        ├───────────────┤                                      »
   q_3: ┤ Ry(-0.019851) ├────────■─────────────────────────────»
        └─────┬───┬─────┘        │                             »
   q_4: ──────┤ H ├──────────────■─────────────────────────────»
              └───┘                                            »
meas_0: ═══════════════════════════════════════════════════════»
                                                               »
meas_1: ═══════════════════════════════════════════════════════»
                                                               »
meas_2: ═══════════════════════════════════════════════════════»
                                                               »
meas_3: ═══════════════════════════════════════════════════════»
                                                               »
meas_4: ═══════════════════════════════════════════════════════»
                                                               »
«          ┌────────────┐                                                 ░ ┌─┐»
«   q_0: ──┤ U(π/2,0,π) ├─────────────────────────────────────────────────░─┤M├»
«        ┌─┴────────────┴┐  ┌────────────┐                                ░ └╥┘»
«   q_1: ┤ Ry(-0.015751) ├──┤ U(π/2,0,π) ├────────────────────────────────░──╫─»
«        └───────────────┘┌─┴────────────┴┐ ┌────────────┐                ░  ║ »
«   q_2: ────────■────────┤ Ry(0.0036205) ├─┤ U(π/2,0,π) ├────────────────░──╫─»
«                │        └───────────────┘┌┴────────────┴┐┌────────────┐ ░  ║ »
«   q_3: ────────■────────────────■────────┤ Ry(0.019873) ├┤ U(π/2,0,π) ├─░──╫─»
«                                 │        └────┬───┬─────┘└────────────┘ ░  ║ »
«   q_4: ─────────────────────────■─────────────┤ H ├─────────────────────░──╫─»
«                                               └───┘                     ░  ║ »
«meas_0: ════════════════════════════════════════════════════════════════════╩═»
«                                                                              »
«meas_1: ══════════════════════════════════════════════════════════════════════»
«                                                                              »
«meas_2: ══════════════════════════════════════════════════════════════════════»
«                                                                              »
«meas_3: ══════════════════════════════════════════════════════════════════════»
«                                                                              »
«meas_4: ══════════════════════════════════════════════════════════════════════»
«                                                                              »
«                    
«   q_0: ────────────
«        ┌─┐         
«   q_1: ┤M├─────────
«        └╥┘┌─┐      
«   q_2: ─╫─┤M├──────
«         ║ └╥┘┌─┐   
«   q_3: ─╫──╫─┤M├───
«         ║  ║ └╥┘┌─┐
«   q_4: ─╫──╫──╫─┤M├
«         ║  ║  ║ └╥┘
«meas_0: ═╬══╬══╬══╬═
«         ║  ║  ║  ║ 
«meas_1: ═╩══╬══╬══╬═
«            ║  ║  ║ 
«meas_2: ════╩══╬══╬═
«               ║  ║ 
«meas_3: ═══════╩══╬═
«                  ║ 
«meas_4: ══════════╩═
«

In [78]:
max_shots = device.configuration().to_dict()['max_shots']
max_shots

32000

In [79]:
trans_qcs = transpile(circuit_list, device, optimization_level = 1, seed_transpiler = 0)

In [80]:
trans_qcs[0].draw()

global phase: π/2
                                                                             »
  ancilla_0 -> 0 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_1 -> 1 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_2 -> 2 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_3 -> 3 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_4 -> 4 ────────────────────────────────────────────────────────────»
                 ┌────┐ ┌────────────┐ ┌────┐                                »
        q_3 -> 5 ┤ √X ├─┤ Rz(1.5509) ├─┤ √X ├────────────────────────────────»
                 └────┘ └────────────┘ └────┘                                »
  ancilla_5 -> 6 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_6 -> 7 ────────────────────────────────────────────────────────────»
                 ┌────┐ ┌────────────┐ ┌────┐                                »
        q_2 -> 8 ┤ √X ├─┤ Rz(1.5746) ├─┤ √X ├────────────────────────────────»
                 └────┘ └────────────┘ └────┘                                »
  ancilla_7 -> 9 ────────────────────────────────────────────────────────────»
                                                                             »
 ancilla_8 -> 10 ────────────────────────────────────────────────────────────»
                 ┌────┐ ┌────────────┐ ┌────┐┌────────┐┌───┐┌─────────┐┌────┐»
       q_1 -> 11 ┤ √X ├─┤ Rz(1.5867) ├─┤ √X ├┤ Rz(-π) ├┤ X ├┤ Rz(π/2) ├┤ √X ├»
                 └────┘ └────────────┘ └────┘└────────┘└─┬─┘└─────────┘└────┘»
 ancilla_9 -> 12 ────────────────────────────────────────┼───────────────────»
                                                         │                   »
ancilla_10 -> 13 ────────────────────────────────────────┼───────────────────»
                 ┌────┐┌──────────────┐┌────┐            │             ┌────┐»
       q_0 -> 14 ┤ √X ├┤ Rz(0.003534) ├┤ √X ├────■───────■───────■─────┤ √X ├»
                 └────┘└──────────────┘└────┘    │               │     └────┘»
ancilla_11 -> 15 ────────────────────────────────┼───────────────┼───────────»
                                               ┌─┴─┐           ┌─┴─┐         »
       q_4 -> 16 ──────────────────────────────┤ X ├───────────┤ X ├─────────»
                                               └───┘           └───┘         »
ancilla_12 -> 17 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_13 -> 18 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_14 -> 19 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_15 -> 20 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_16 -> 21 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_17 -> 22 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_18 -> 23 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_19 -> 24 ────────────────────────────────────────────────────────────»
                                

In [81]:
trans_qcs[-1].draw()

global phase: 2π
                                                                       »
  ancilla_0 -> 0 ──────────────────────────────────────────────────────»
                                                                       »
  ancilla_1 -> 1 ──────────────────────────────────────────────────────»
                                                                       »
  ancilla_2 -> 2 ──────────────────────────────────────────────────────»
                                                                       »
  ancilla_3 -> 3 ──────────────────────────────────────────────────────»
                                                                       »
  ancilla_4 -> 4 ──────────────────────────────────────────────────────»
                 ┌────┐┌──────────────┐┌────┐                  ┌────┐  »
        q_0 -> 5 ┤ √X ├┤ Rz(0.003534) ├┤ √X ├────────────■─────┤ √X ├──»
                 └────┘└──────────────┘└────┘            │     └────┘  »
  ancilla_5 -> 6 ────────────────────────────────────────┼─────────────»
                                                         │             »
  ancilla_6 -> 7 ────────────────────────────────────────┼─────────────»
                 ┌────┐ ┌────────────┐ ┌────┐┌────────┐┌─┴─┐┌─────────┐»
        q_1 -> 8 ┤ √X ├─┤ Rz(1.5867) ├─┤ √X ├┤ Rz(-π) ├┤ X ├┤ Rz(π/2) ├»
                 └────┘ └────────────┘ └────┘└────────┘└───┘└─────────┘»
  ancilla_7 -> 9 ──────────────────────────────────────────────────────»
                                                                       »
 ancilla_8 -> 10 ──────────────────────────────────────────────────────»
                 ┌────┐ ┌────────────┐ ┌────┐                          »
       q_2 -> 11 ┤ √X ├─┤ Rz(1.5746) ├─┤ √X ├──────────────────────────»
                 └────┘ └────────────┘ └────┘                          »
 ancilla_9 -> 12 ──────────────────────────────────────────────────────»
                                                                       »
ancilla_10 -> 13 ──────────────────────────────────────────────────────»
                 ┌────┐ ┌────────────┐ ┌────┐┌────────┐     ┌─────────┐»
       q_3 -> 14 ┤ √X ├─┤ Rz(3.1217) ├─┤ √X ├┤ Rz(-π) ├──■──┤ Rz(π/2) ├»
                 └────┘ └────────────┘ └────┘└────────┘  │  └─────────┘»
ancilla_11 -> 15 ────────────────────────────────────────┼─────────────»
                                                       ┌─┴─┐           »
       q_4 -> 16 ──────────────────────────────────────┤ X ├───────────»
                                                       └───┘           »
ancilla_12 -> 17 ──────────────────────────────────────────────────────»
                                                                       »
ancilla_13 -> 18 ──────────────────────────────────────────────────────»
                                                                       »
ancilla_14 -> 19 ──────────────────────────────────────────────────────»
                                                                       »
ancilla_15 -> 20 ──────────────────────────────────────────────────────»
                                                                       »
ancilla_16 -> 21 ──────────────────────────────────────────────────────»
                                                                       »
ancilla_17 -> 22 ──────────────────────────────────────────────────────»
                                                                       »
ancilla_18 -> 23 ──────────────────────────────────────────────────────»
                                                                       »
ancilla_19 -> 24 ──────────────────────────────────────────────────────»
                                                                       »
ancilla_20 -> 25 ──────────────────────────────────────────────────────»
                                                                       »
ancilla_21 -> 26 ──────────────────────────────────────────────────────»
                                         

In [82]:
for qc in trans_qcs:
    print(qc.count_ops())

OrderedDict([('sx', 14), ('rz', 14), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 16), ('rz', 14), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 16), ('rz', 16), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 16), ('rz', 15), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 16), ('rz', 15), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 16), ('rz', 15), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 16), ('rz', 13), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 16), ('rz', 14), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 16), ('rz', 14), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 18), ('rz', 15), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 18), ('rz', 15), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 18), ('rz', 16), ('cx', 5), ('measure', 5), ('barrier', 1)])
OrderedDict([('sx', 18), ('r

In [83]:
trans_qcs[0].draw()

global phase: π/2
                                                                             »
  ancilla_0 -> 0 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_1 -> 1 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_2 -> 2 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_3 -> 3 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_4 -> 4 ────────────────────────────────────────────────────────────»
                 ┌────┐ ┌────────────┐ ┌────┐                                »
        q_3 -> 5 ┤ √X ├─┤ Rz(1.5509) ├─┤ √X ├────────────────────────────────»
                 └────┘ └────────────┘ └────┘                                »
  ancilla_5 -> 6 ────────────────────────────────────────────────────────────»
                                                                             »
  ancilla_6 -> 7 ────────────────────────────────────────────────────────────»
                 ┌────┐ ┌────────────┐ ┌────┐                                »
        q_2 -> 8 ┤ √X ├─┤ Rz(1.5746) ├─┤ √X ├────────────────────────────────»
                 └────┘ └────────────┘ └────┘                                »
  ancilla_7 -> 9 ────────────────────────────────────────────────────────────»
                                                                             »
 ancilla_8 -> 10 ────────────────────────────────────────────────────────────»
                 ┌────┐ ┌────────────┐ ┌────┐┌────────┐┌───┐┌─────────┐┌────┐»
       q_1 -> 11 ┤ √X ├─┤ Rz(1.5867) ├─┤ √X ├┤ Rz(-π) ├┤ X ├┤ Rz(π/2) ├┤ √X ├»
                 └────┘ └────────────┘ └────┘└────────┘└─┬─┘└─────────┘└────┘»
 ancilla_9 -> 12 ────────────────────────────────────────┼───────────────────»
                                                         │                   »
ancilla_10 -> 13 ────────────────────────────────────────┼───────────────────»
                 ┌────┐┌──────────────┐┌────┐            │             ┌────┐»
       q_0 -> 14 ┤ √X ├┤ Rz(0.003534) ├┤ √X ├────■───────■───────■─────┤ √X ├»
                 └────┘└──────────────┘└────┘    │               │     └────┘»
ancilla_11 -> 15 ────────────────────────────────┼───────────────┼───────────»
                                               ┌─┴─┐           ┌─┴─┐         »
       q_4 -> 16 ──────────────────────────────┤ X ├───────────┤ X ├─────────»
                                               └───┘           └───┘         »
ancilla_12 -> 17 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_13 -> 18 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_14 -> 19 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_15 -> 20 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_16 -> 21 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_17 -> 22 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_18 -> 23 ────────────────────────────────────────────────────────────»
                                                                             »
ancilla_19 -> 24 ────────────────────────────────────────────────────────────»
                                

In [84]:
trans_qcs[50].draw()

global phase: π/4
                                                                        »
  ancilla_0 -> 0 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_1 -> 1 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_2 -> 2 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_3 -> 3 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_4 -> 4 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_5 -> 5 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_6 -> 6 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_7 -> 7 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_8 -> 8 ───────────────────────────────────────────────────────»
                                                                        »
  ancilla_9 -> 9 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_10 -> 10 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_11 -> 11 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_12 -> 12 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_13 -> 13 ───────────────────────────────────────────────────────»
                   ┌────┐  ┌──────────────┐    ┌────┐                   »
       q_0 -> 14 ──┤ √X ├──┤ Rz(0.003534) ├────┤ √X ├────────────────■──»
                   └────┘  └──────────────┘    └────┘                │  »
ancilla_14 -> 15 ────────────────────────────────────────────────────┼──»
                   ┌────┐   ┌────────────┐     ┌────┐    ┌────────┐┌─┴─┐»
       q_1 -> 16 ──┤ √X ├───┤ Rz(1.5867) ├─────┤ √X ├────┤ Rz(-π) ├┤ X ├»
                   └────┘   └────────────┘     └────┘    └────────┘└───┘»
ancilla_15 -> 17 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_16 -> 18 ───────────────────────────────────────────────────────»
                 ┌────────┐     ┌────┐     ┌────────────┐  ┌────┐       »
       q_2 -> 19 ┤ Rz(-π) ├─────┤ √X ├─────┤ Rz(3.1378) ├──┤ √X ├────■──»
                 └────────┘     └────┘     └────────────┘  └────┘  ┌─┴─┐»
       q_4 -> 20 ──────────────────────────────────────────────────┤ X ├»
                                                                   └───┘»
ancilla_17 -> 21 ───────────────────────────────────────────────────────»
                   ┌────┐   ┌────────────┐     ┌────┐                   »
       q_3 -> 22 ──┤ √X ├───┤ Rz(1.5509) ├─────┤ √X ├───────────────────»
                   └────┘   └────────────┘     └────┘                   »
ancilla_18 -> 23 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_19 -> 24 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_20 -> 25 ───────────────────────────────────────────────────────»
                                                                        »
ancilla_21 -> 26 ───────────────────────────────────────────

In [85]:
#11/18 map to qubits 16,14,11,8,5
job = execute(circuit_list, backend = device, optimization_level = 1, shots = max_shots, seed_transpiler = 0)
print(job.job_id())

6377ebfc4a1eb599ba1b3668


In [86]:
job_monitor(job)

Job Status: job has successfully run


In [87]:
# job = device.retrieve_job(job_id='')

In [88]:
result = job.result()

In [90]:
len(result.get_counts())

100

In [104]:
import json
with open('LiH_toronto_nocut.json', 'w') as f:
    json.dump(result.get_counts(), f)

In [98]:
per_miti_results = []
for i in range(0, 4):
    temp_list = []
    for j in range(0, 25):
        filter_per_miti = norm_dict(filter_results(result.get_counts()[i*25 + j], [0]))
        temp_list.append(filter_per_miti)
    per_miti_results.append(temp_list)

In [99]:
len(per_miti_results)

4

In [102]:
for j in range(0, 4):
    final_expect_val = 0
    for i in range(0, len(pauli_commute)):
        group = pauli_commute[i]
        for Pauli_tuple in group:
            coeff = Pauli_tuple[1]
            final_expect_val += coeff * evaluation(per_miti_results[j][i], shots = total_counts(per_miti_results[j][i]), Pauli = Pauli_tuple[0])  
    print(final_expect_val)

(-1.0320384574679138+0j)
(-0.8265571718461543+0j)
(-0.9991044528351548+0j)
(-1.0258811261473502+0j)
